In [2]:
import pandas as pd 
import plotly.express as px
import mibian as mb
import yfinance
from datetime import datetime

In [157]:
raw_options = pd.read_csv("rawtrades.csv") #import data
raw_options["Underlying"] = [yfinance.Ticker(i).history(period="6mo").loc[d][["High", "Low"]].mean() for i,d in zip(ticker_names,ticker_dates)] #get mean price of underlying

In [169]:
vol = [mb.BS([i,j,1,k], callPrice=l).impliedVolatility if t=="C" else mb.BS([i,j,1,k], putPrice=l).impliedVolatility for i,j,k,l,t in zip(raw_options["Underlying"], raw_options["Strike"], raw_options["DTE"], (raw_options["Cost"]/100), raw_options["Type"])] # get implied volaility for greek calculations

In [180]:
theta = [mb.BS([i,j,1,k], volatility=l).callTheta if t=="C" else mb.BS([i,j,1,k], volatility=l).putTheta for i,j,k,l,t in zip(raw_options["Underlying"], raw_options["Strike"], raw_options["DTE"], raw_options["ImpliedVol"], raw_options["Type"])]

In [199]:
adjThe = [th if a=="BTO" else -th for th,a in zip(theta,raw_options["Action"])]

In [200]:
raw_options["Theta"] = adjThe

In [203]:
summed = raw_options.groupby(by="Trade ID").sum()

In [207]:
summed["PL"] = summed["Credit"] + summed["Debit"]

In [211]:
summed = summed[summed["PL"] > -200]

In [3]:
px.scatter(summed, x="PL", y="Delta")

NameError: name 'summed' is not defined

In [220]:
summed

,Strike,DTE,Credit,Debit,Cost,Underlying,ImpliedVol,Delta,Theta,PL
Trade ID,,,,,,,,,,
1,92.0,132,49.68,-39.56,89.24,94.400000,499.259949,0.297695,0.056329,10.12
2,46.0,50,73.84,-72.14,145.98,46.480001,70.610046,-0.868179,0.033205,1.70
3,40.0,80,319.28,-297.71,616.99,45.374999,294.731140,-0.682616,0.095767,21.57
4,42.0,85,215.29,-285.71,501.00,43.400001,189.601898,-0.833371,0.061052,-70.42
5,260.0,178,119.53,-257.42,376.95,262.205333,269.874573,1.159561,0.687124,-137.89
6,92.0,336,301.13,-279.08,580.21,98.480000,730.773926,1.111354,0.062844,22.05
7,40.0,134,305.28,-335.71,640.99,39.710001,186.403275,-0.852454,0.045753,-30.43
8,36.0,316,374.28,-380.71,754.99,39.710001,192.018509,-0.632668,0.027698,-6.43
9,52.0,56,60.29,-81.71,142.00,54.090919,79.757690,-0.676293,0.038882,-21.42
